In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#metrics
from statistics import mean
from sklearn.metrics import accuracy_score 
from sklearn.metrics import explained_variance_score as evs
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

Matplotlib is building the font cache; this may take a moment.


In [11]:
simd=pd.read_csv('../data/combined_dataset_cleaned.csv')

In [12]:
simd.head()

,Unnamed: 0,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_age_population_revised,Income_rate,Income_count,Employment_rate,Employment_count,...,PT_GP,PT_Post,PT_retail,crime_count,crime_rate,overcrowded_count,nocentralheat_count,overcrowded_rate,nocentralheat_rate,year
0,0,S01006506,Culter,Aberdeen City,904.0,605.0,0.07,60.0,0.07,40.0,...,8.437832,5.987087,5.711433,8.00642939150402,88.5666968086728,87.0,10.0,0.102113,0.011737,2016
1,1,S01006507,Culter,Aberdeen City,830.0,491.0,0.07,60.0,0.05,25.0,...,8.331833,7.262817,6.794300,4.00321469575201,48.2315023584579,85.0,4.0,0.101675,0.004785,2016
2,2,S01006508,Culter,Aberdeen City,694.0,519.0,0.05,30.0,0.03,15.0,...,7.853631,5.827924,5.251454,4.00321469575201,57.6832088725073,31.0,8.0,0.048212,0.012442,2016
3,3,S01006509,Culter,Aberdeen City,573.0,354.0,0.05,30.0,0.06,20.0,...,7.434491,8.311862,8.444698,*,*,42.0,6.0,0.072414,0.010345,2016
4,4,S01006510,Culter,Aberdeen City,676.0,414.0,0.10,70.0,0.07,30.0,...,5.141013,6.627376,6.619130,12.009644087256,177.657456912071,50.0,7.0,0.086655,0.012132,2016


In [13]:
simd.isnull().sum()

Unnamed: 0                        0
Data_Zone                         0
Intermediate_Zone                 0
Council_area                      0
Total_population                  0
Working_age_population_revised    0
Income_rate                       2
Income_count                      0
Employment_rate                   2
Employment_count                  0
CIF                               0
ALCOHOL                           0
DRUG                              0
SMR                               0
DEPRESS                           0
LBWT                              0
EMERG                             0
Attendance                        2
Attainment                        1
Noquals                           0
NEET                              3
HESA                              0
drive_petrol                      0
drive_GP                          0
drive_PO                          0
drive_primary                     0
drive_retail                      0
drive_secondary             